# Logs analysis

In [1]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import re

## List all logs files

In [2]:
import glob
files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(10,
 ['../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.2/sampling_method=CD-1/momentum=1/1550421566.1091068/events.out.tfevents.1550421571.paulo-notebook',
  '../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBM/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.1/sampling_method=CD-1/momentum=1/1550438898.2907436/events.out.tfevents.1550438902.paulo-notebook',
  '../results/logs/kfold=4/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=500/regularization=NoRegularization-0.0/learning_rate=ConstantLearningRate-0.2/sampling_method=CD-1/momentum=1/1550424962.7375395/events.out.tfevents.1550424967.paulo-notebook'])

## Read a file

In [3]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    #train = pd.DataFrame(ea.Scalars('measure/evaluate/train'))
    #validation = pd.DataFrame(ea.Scalars('measure/evaluate/validation'))
    
    del ea

    return dataframes

dfs = extract_information(file)
dfs.head()

#train, validation = extract_information(file)
#train.head()

,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,details/measure/expectation/normalized/evaluate-3/train,details/measure/expectation/normalized/evaluate-3/validation,details/measure/expectation/normalized/evaluate-4/train,details/measure/expectation/normalized/evaluate-4/validation,details/measure/expectation/normalized/evaluate-5/train,details/measure/expectation/normalized/evaluate-5/validation,measure/evaluate/expectation/normalized/RMSE_train,measure/evaluate/expectation/normalized/RMSE_validation,measure/evaluate/expectation/normalized/train,measure/evaluate/expectation/normalized/validation
0,1.550422e+09,1,0.122436,5.907176,5.125681,-0.037394,-4.359916,-0.001174,3.815010,0.331139,...,0.025016,0.015789,0.019750,0.015789,0.009875,0.002632,0.130110,0.130313,0.009655,0.006579
1,1.550422e+09,2,0.120661,5.980908,5.227418,-0.037773,-4.415276,-0.000415,3.472679,0.337722,...,0.023700,0.013158,0.023700,0.018421,0.015800,0.015789,0.129915,0.130169,0.012618,0.008772
2,1.550422e+09,3,0.119760,5.926267,4.789822,-0.039035,-4.446284,-0.000905,3.337064,0.323239,...,0.016458,0.023684,0.023041,0.015789,0.009875,0.007895,0.130132,0.130053,0.009326,0.010526
3,1.550422e+09,4,0.119144,5.915076,5.400952,-0.039910,-4.456356,0.000332,3.301514,0.319289,...,0.021066,0.005263,0.025016,0.007895,0.008558,0.013158,0.130038,0.130428,0.010753,0.004825
4,1.550422e+09,5,0.118754,5.968400,5.378855,-0.040439,-4.443341,0.000478,3.188940,0.319947,...,0.010533,0.015789,0.025675,0.013158,0.023700,0.018421,0.129980,0.130111,0.011630,0.009649


In [4]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    columns = {match.groups()[0]: match.groups()[1] for match in matches}
    columns['path'] = test_str
    return columns

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [5]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 10
2 of 10
3 of 10
4 of 10
5 of 10
6 of 10
7 of 10
8 of 10
9 of 10
10 of 10


In [6]:
data.index.name = 'evaluate'
data.head(5)

,wall_time,step,measure/reconstruction/error,measure/activation/mean,measure/activation/std,measure/parameters/W/mean,measure/parameters/b_h/mean,measure/parameters/b_v/mean,measure/reconstruction_1/hamming,details/measure/top-1/evaluate-0/train,...,kfold-intern,batch_size,class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum,path
evaluate,,,,,,,,,,,,,,,,,,,,,
0,1.550422e+09,1,0.122436,5.907176,5.125681,-0.037394,-4.359916,-0.001174,3.815010,0.331139,...,0,10,RBMCF,702,500,NoRegularization-0.0,ConstantLearningRate-0.2,CD-1,1,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
1,1.550422e+09,2,0.120661,5.980908,5.227418,-0.037773,-4.415276,-0.000415,3.472679,0.337722,...,0,10,RBMCF,702,500,NoRegularization-0.0,ConstantLearningRate-0.2,CD-1,1,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
2,1.550422e+09,3,0.119760,5.926267,4.789822,-0.039035,-4.446284,-0.000905,3.337064,0.323239,...,0,10,RBMCF,702,500,NoRegularization-0.0,ConstantLearningRate-0.2,CD-1,1,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
3,1.550422e+09,4,0.119144,5.915076,5.400952,-0.039910,-4.456356,0.000332,3.301514,0.319289,...,0,10,RBMCF,702,500,NoRegularization-0.0,ConstantLearningRate-0.2,CD-1,1,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
4,1.550422e+09,5,0.118754,5.968400,5.378855,-0.040439,-4.443341,0.000478,3.188940,0.319947,...,0,10,RBMCF,702,500,NoRegularization-0.0,ConstantLearningRate-0.2,CD-1,1,kfold=2/kfold-intern=0/batch_size=10/class=RBM...


In [7]:
#data.to_csv('extracted_log.csv')
data.to_csv('extracted_log-rbm_rbm-cf_final.csv')